# Projeto 03

## Importação das bibliotecas

In [173]:
import pandas as pd
import numpy as np
import bibtexparser
import yaml
import requests
import json
import pyodbc
import sqlalchemy as sa
import urllib
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
import hashlib
import re

## Carregando arquivo de configuração YAML

In [174]:
with open(r"C:\Users\valdi\Documents\Python\Projeto 3\Config\config_final.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print("Read successful")
print(config)

Read successful
{'search': 'NoSQL', 'type_save': 'json, csv, yaml, parquet', 'filter_title': 'data', 'filter_year': 2020, 'jcs_origem': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Source\\jcs_2020.csv', 'scimago_origem': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Source\\scimagojr 2020.csv', 'acm_origem': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Source\\ACM.bib', 'ieee_origem': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Source\\IEE.bib', 'science_direct_origem': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Source\\ScienceDirect.bib', 'csv_dest': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Export\\export_final.csv', 'json_dest': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Export\\export_final.json', 'yaml_dest': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Export\\export_final.yaml', 'parquet_gzip_dest': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Export\\export_final.gzip', 'xml_dest': 'C:\\Users\\valdi\\Documents\\Python\\Projeto 3\\Expo

## Funções uteis

In [175]:
#Função responsável por gerar o hash
def hash(value):
    return int(hashlib.sha256(value.encode('utf-8')).hexdigest(), 16) % 10**8

#Função responsável por tratar o valor da coluna onde é alfanumérica, 
# removendo também os caracteres especiais antes de jogar na função de hash
def str_key(value):
    if value != ' ': 
        return hash(''.join(str(world).lower() for world in str(value) if str(world).isalnum()))
    return np.nan

#Função responsável por tratar a coluna nula e criando a coluna com prefixo id_,
# já passando as informações para a função str_key
def make_str_key(df: pd.DataFrame, coluna: str)-> pd.DataFrame: 
    df[coluna] = df[coluna].fillna(value=' ')

    coluna_key = "id_" + coluna 
    df[coluna_key] = df[coluna].apply(str_key) 
                                                
    return df

#Função responsável por realizar um case para unificar os dados de 
# colunas com nomes duplicados no momentos do join
def unificacao(coluna: str, df: pd.DataFrame,  case: int) -> pd.DataFrame:
    coluna_y = coluna + '_y'
    coluna_z = coluna + '_z'
    
    if case == 2:
        coluna_y = coluna + '_y'
        df[coluna] = np.where(df[coluna] == ' ', df[coluna_y], df[coluna])
    
    elif case == 3:
        x = 1
        df[coluna] = np.where(df[coluna_y] != ' ', df[coluna_y],
                      np.where(df[coluna] != ' ', df[coluna], 
                               df[coluna_z]))
    
    return df

#Função responsável por realizar a alteração em uma lista de colunas que forem nulas para ' '
def alter_null(colunas: list, df: pd.DataFrame) -> pd.DataFrame:
    for coluna in colunas:
        df[coluna] = df[coluna].fillna(' ')
    
    return df

## API

### Leitura com a API do IEEE

In [176]:
#Jogando para uma var a palavra a config 
search = config["search"]

#Realizando a requisição
request_ieee = requests.get(f"http://ieeexploreapi.ieee.org/api/v1/search/articles?querytext={search}&format=json&apikey=cf6zh8aabdqy6zbe5gpqf3f7")

#Convertendo para json para que o resultado da requisição possa 
#ser lido em um dataframe
request_ieee = request_ieee.json()

#Exibindo o total de registros e total pesquisado, resultantes da requisição
print("Total de registro: ", request_ieee["total_records"])
print("Total pesquisado: ", request_ieee["total_searched"])

#Passando para uma var apenas os artigos
api_ieee = request_ieee['articles']

#Lendo os dados no dataframe
df_ieee_api = pd.DataFrame(api_ieee)

#Trantando coluna de autores que estava em formato de dicionario, com vários atores,
#foi realizado o apply e explode para trazer os dados contidos no interior dos dicionarios,
#tornando eles em colunas.
df_ieee_api = pd.concat([df_ieee_api.drop(['authors'], axis=1), 
                         df_ieee_api['authors'].apply(pd.Series)], axis=1)

df_ieee_api = df_ieee_api.explode('authors')

df_ieee_api = pd.concat([df_ieee_api.drop(['authors'], axis=1), 
                         df_ieee_api['authors'].apply(pd.Series)], axis=1)

#Resetando index
df_ieee_api.set_index('doi', inplace=True)
df_ieee_api.reset_index(inplace=True)

#Removendo '-' da coluna issn
df_ieee_api['issn'].replace('-', '', regex=True, inplace=True)


#Selecionando as colunas para evitar possiveis retornos inesperados da api
df_ieee_api = pd.DataFrame(df_ieee_api, columns=['doi', 'title','publisher','isbn','rank','content_type',
                'abstract','article_number','pdf_url','html_url','abstract_url','publication_title','conference_location'
                ,'conference_dates','publication_number','is_number','publication_year','publication_date'     
                ,'start_page','end_page','citing_paper_count','citing_patent_count'  
                ,'issn','issue','volume','affiliation','authorUrl','id'            
                ,'full_name','author_order'])         

#Renomeando coluna
df_ieee_api.rename(columns={'publication_year': 'year', 'id':'id_author', 'full_name': 'author'}, inplace=True)

df_ieee_api.head(2)

Total de registro:  1306
Total pesquisado:  5791710


,doi,title,publisher,isbn,rank,content_type,abstract,article_number,pdf_url,html_url,...,citing_paper_count,citing_patent_count,issn,issue,volume,affiliation,authorUrl,id_author,author,author_order
0,10.1109/IC3TSN.2017.8284495,Security analysis of unstructured data in NOSQ...,IEEE,978-1-5386-0628-5,1,Conferences,NoSQL databases systems are non-relational dat...,8284495,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,https://ieeexplore.ieee.org/document/8284495/,...,8,0,NaN,NaN,NaN,Jaypee Institute of Information Technology,https://ieeexplore.ieee.org/author/37086283795,3.708628e+10,Jitender Kumar,1
1,10.1109/IC3TSN.2017.8284495,Security analysis of unstructured data in NOSQ...,IEEE,978-1-5386-0628-5,1,Conferences,NoSQL databases systems are non-relational dat...,8284495,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,https://ieeexplore.ieee.org/document/8284495/,...,8,0,NaN,NaN,NaN,Jaypee Institute of Information Technology,https://ieeexplore.ieee.org/author/37086279979,3.708628e+10,Varsha Garg,2


### Leitura com a API do Science Direct

In [177]:
request_sc = requests.get(f"https://api.elsevier.com/content/search/sciencedirect?query={search}&apiKey=7f59af901d2d86f78a1fd60c1bf9426a")

request_sc = request_sc.json()

#Exibindo o total de registros, resultantes da requisição
print("Total de registros: ", request_sc['search-results']['opensearch:totalResults'])

api_science_direct = request_sc['search-results']['entry']

#Lendo os dados no dataframe
df_sc_api = pd.DataFrame(api_science_direct)

#Trantando coluna de autores que estava em formato de dicionario, com vários atores,
#foi realizado o apply e explode para trazer os dados contidos no interior dos dicionarios,
#tornando eles em colunas.
df_sc_api = pd.concat([df_sc_api.drop(['authors'], axis=1), 
                         df_sc_api['authors'].apply(pd.Series)], axis=1)

df_sc_api = df_sc_api.explode('author')

df_sc_api = pd.concat([df_sc_api.drop(['author'], axis=1), 
                         df_sc_api['author'].apply(pd.Series)], axis=1)

#Renomeando coluna
df_sc_api.rename(columns={'load-date': 'load date', 'prism:url':'url', 'dc:title': 'title', 
    'dc:creator': 'creator', 'prism:publicationName':'publication name','prism:doi':'doi',
    'prism:volume': 'volume', 'prism:coverDate':'cover date', 'prism:endingPage': 'ending page', 
    'prism:startingPage':'starting page', '$':'author'}, inplace=True)

#Selecionando as colunas para evitar possiveis retornos inesperados da api
df_sc_api = pd.DataFrame(df_sc_api, columns=['doi', 'load date', 'url', 'title', 'creator', 'publication name'
                                'volume', 'cover date', 'starting page', 'pii', 'ending page', 'author'])

#Resetando index
df_sc_api.set_index('doi', inplace=True)
df_sc_api.reset_index(inplace=True)

df_sc_api.head(2)

Total de registros:  2275


,doi,load date,url,title,creator,publication namevolume,cover date,starting page,pii,ending page,author
0,10.1016/j.datak.2022.102089,2022-10-15T00:00:00.000Z,https://api.elsevier.com/content/article/pii/S...,A workload-driven method for designing aggrega...,Liu Chen,NaN,2022-11-30,102089,S0169023X22000805,NaN,Liu Chen
1,10.1016/j.datak.2022.102089,2022-10-15T00:00:00.000Z,https://api.elsevier.com/content/article/pii/S...,A workload-driven method for designing aggrega...,Liu Chen,NaN,2022-11-30,102089,S0169023X22000805,NaN,Ali Davoudian


## Leitura dos arquivos CSV

### Arquivo JCS

In [178]:
#Leitura e padronização do arquivo JCS
df_jcs = pd.read_csv(config["jcs_origem"], sep=';')

#Guardando os dados brutos
df_jcs_raw = df_jcs.copy()

#Dropando colunas não utilizadas
df_jcs.drop(["Rank", "Unnamed: 3", "Unnamed: 6", "Unnamed: 7"], axis=1, inplace=True)

#Renomeando coluna
df_jcs.rename(columns={'Full Journal Title':'title', 'Title':'title'}, inplace=True)

#Removendo as linhas duplicadas
df_jcs.drop_duplicates(keep='first',inplace=True)

#Criando coluna hash baseado na coluna title
df_jcs = make_str_key(df_jcs, 'title')

df_jcs.head(2)

,title,Total Cites,Journal Impact Factor,Eigenfactor Score,id_title
0,CA-A CANCER JOURNAL FOR CLINICIANS,"55,868",508.702,0.10514,70032868
1,NATURE REVIEWS MOLECULAR CELL BIOLOGY,"58,477",94.444,0.07548,34291073


### Arquivo SCIMAGO

In [179]:
#Leitura e padronização do arquivo SCIMAGO
df_scimago = pd.read_csv(config["scimago_origem"], sep=';', 
        quotechar='"', low_memory=False)

#Guardando os dados brutos
df_scimago_raw = df_scimago.copy()

#Dropando colunas não utilizadas
df_scimago.drop(["Rank"], axis=1, inplace=True)

#Renomeando coluna
df_scimago.rename(columns={'Issn':'issn_full','Title':'title', 'Type':'type_publication', 
                           'Publisher':'publisher'}, inplace=True)

#Criando uma coluna nova para receber os issn's existentes dentro do issn_full
df_scimago['issn'] = df_scimago['issn_full'].str.split(', ')
df_scimago = df_scimago.explode('issn')

#Removendo as linhas duplicadas
df_scimago.drop_duplicates(keep='first',inplace=True)

#Criando coluna hash baseado na coluna title
df_scimago = make_str_key(df_scimago, 'title')

df_scimago.head(2)

,Sourceid,title,type_publication,issn_full,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,...,Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,publisher,Coverage,Categories,issn,id_title
0,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,3452,...,80,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),15424863,70032868
0,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,3452,...,80,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),00079235,70032868


## Join e tratamento dos arquivos csv

In [180]:
#Realizando join entre o arquivo SCIMAGO e JCS
df_join = pd.merge(df_scimago, df_jcs, on=["id_title"], how="outer")

#Renomeando coluna
df_join.rename(columns={'title_x':'title'}, inplace=True)

#Unificando a coluna title
df_join['title'] = df_join['title'].fillna(' ')

df_join = unificacao('title', df_join, 2)

#Dropando as colunas title que não serão mais uteis
df_join.drop(["title_y", "id_title"], axis=1, inplace=True)        
    
df_join.head(2)

,Sourceid,title,type_publication,issn_full,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,...,Ref. / Doc.,Country,Region,publisher,Coverage,Categories,issn,Total Cites,Journal Impact Factor,Eigenfactor Score
0,28773.0,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,168.0,47.0,119.0,3452.0,...,"73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),15424863,"55,868",508.702,0.10514
1,28773.0,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,168.0,47.0,119.0,3452.0,...,"73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),00079235,"55,868",508.702,0.10514


## Leitura de Arquivos BibTex

### Arquivo ACM 

In [181]:
#Leitura e padronização do arquivo
with open(config["acm_origem"], encoding="utf8") as bibtex_file_acm:
    bibtex_database_acm = bibtexparser.load(bibtex_file_acm)   
    
df_acm = pd.DataFrame(bibtex_database_acm.entries)

#Guardando os dados brutos
df_acm_raw = df_acm.copy()

#Renomeando coluna
df_acm.rename(columns={'ENTRYTYPE':'type_publication'}, inplace=True)

#Removendo '-' da coluna issn
df_acm['issn'].replace('-', '', regex=True, inplace=True)

df_acm.head(2)

,series,location,keywords,numpages,pages,booktitle,abstract,doi,url,address,...,articleno,month,journal,issn,number,volume,issue_date,note,edition,editor
0,BDE 2019,"Hong Kong, Hong Kong","Sentiment analysis, Big data quality metrics, ...",8,36–43,Proceedings of the 2019 International Conferen...,"In a world increasingly connected, and in whic...",10.1145/3341620.3341629,https://doi.org/10.1145/3341620.3341629,"New York, NY, USA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SITA'20,"Rabat, Morocco","Data Quality evaluation, Data Quality, Quality...",6,NaN,Proceedings of the 13th International Conferen...,"In recent years, as more and more data sources...",10.1145/3419604.3419803,https://doi.org/10.1145/3419604.3419803,"New York, NY, USA",...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Arquivo IEEE 

In [182]:
#Leitura e padronização do arquivo
with open(config["ieee_origem"], encoding="utf8") as bibtex_file_iee:
    bibtex_database_iee = bibtexparser.load(bibtex_file_iee)     
    
df_iee = pd.DataFrame(bibtex_database_iee.entries)

#Guardando os dados brutos
df_iee_raw = df_iee.copy()

#Renomeando coluna
df_iee.rename(columns={'ENTRYTYPE':'type_publication'}, inplace=True)

#Removendo '-' da coluna issn
df_iee['issn'].replace('-', '', regex=True, inplace=True)

df_iee.head(2)

,month,issn,doi,keywords,abstract,pages,number,volume,year,title,booktitle,author,type_publication,ID,journal,url,isbn,publisher
0,June,,10.1109/BigDataCongress.2017.73,Big Data;Optimization;Data models;Quality asse...,"In the Big Data Era, data is the core for any ...",498-501,,,2017,Big Data Pre-Processing: Closing the Data Qual...,2017 IEEE International Congress on Big Data (...,"Taleb, Ikbal and Serhani, Mohamed Adel",inproceedings,8029366,NaN,NaN,NaN,NaN
1,Nov,,10.1109/WISA.2017.29,Big Data;Data integrity;Power grids;History;Re...,Since a low-quality data may influence the eff...,289-292,,,2017,A Big Data Framework for Electric Power Data Q...,2017 14th Web Information Systems and Applicat...,"Liu, He and Huang, Fupeng and Li, Han and Liu,...",inproceedings,8332632,NaN,NaN,NaN,NaN


### Arquivo ScienceDirect

In [183]:
#Leitura e padronização do arquivo
with open(config["science_direct_origem"], encoding="utf8") as bibtex_file_ScienceDirect:
    bibtex_database_ScienceDirect = bibtexparser.load(bibtex_file_ScienceDirect)     
    
df_ScienceDirect = pd.DataFrame(bibtex_database_ScienceDirect.entries)

#Guardando os dados brutos
df_ScienceDirect_raw = df_ScienceDirect.copy()

#Renomeando coluna
df_ScienceDirect.rename(columns={'ENTRYTYPE':'type_publication'}, inplace=True)

#Removendo '-' da coluna issn
df_ScienceDirect['issn'].replace('-', '', regex=True, inplace=True)

df_ScienceDirect.head(2)

,abstract,keywords,author,url,doi,issn,note,year,pages,volume,...,type_publication,ID,number,booktitle,publisher,series,editor,isbn,address,edition
0,"Beyond the hype of Big Data, something within ...","Data Quality, Big Data, Measurement, Quality-i...",Jorge Merino and Ismael Caballero and Bibiano ...,https://www.sciencedirect.com/science/article/...,https://doi.org/10.1016/j.future.2015.11.024,0167739X,Modeling and Management for Big Data Analytics...,2016,123-130,63,...,article,MERINO2016123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Xintao Liu and Min Chen and Christophe Claramu...,https://www.sciencedirect.com/science/article/...,https://doi.org/10.1016/j.xinn.2022.100279,26666758,NaN,2022,100279,3,...,article,LIU2022100279,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## União e join entre os arquivos bibtex, csv e api's

In [184]:
#Concatenando os dataframes acm, iee e ScienceDirect
df_bib = pd.concat([df_acm, df_iee, df_ScienceDirect], ignore_index=True)

#Efetuando o join com a coluna issn
df_final = pd.merge(df_join, df_bib, on=["issn"], how="right")

#Concatenando com os dados da api do IEEE e ScienceDirect
df_final = pd.concat([df_final, df_ieee_api, df_sc_api])

## Tratamento Pós união dos dados

In [185]:
#Renomeando coluna
df_final.rename(columns={'title':'title_z', 'title_x':'title', 'type_publication_x':'type_publication',
                        'publisher':'publisher_z', 'publisher_x':'publisher'}, inplace=True)

#Unificando a coluna title, type_publication e publisher
colunas = ['title', 'title_y', 'type_publication', 'publisher', 'publisher_y', 'issn_full']
df_final = alter_null(colunas, df_final)

df_final = unificacao('title', df_final, 3)
df_final = unificacao('type_publication', df_final, 2)
df_final = unificacao('publisher', df_final, 3)

#Unificando a coluna issn e issn_full (criada na leitura de um dos arquivos csv)
df_final['issn'] = np.where(df_final['issn_full'] != ' ', df_final['issn_full'], df_final['issn'])

#Criando coluna hash baseado na coluna title e issn para utilizar no drop duplicates
df_final = make_str_key(df_final, 'title')
df_final = make_str_key(df_final, 'issn')

#Removendo as linhas duplicadas
df_final.drop_duplicates(subset=['id_title', 'id_issn'], keep='first',inplace=True)

#Dropando as colunas title que não serão mais uteis
df_final.drop(["title_y", "title_z", "type_publication_y", "publisher_y", "publisher_z", 
               "issn_full", "id_issn","id_title"], axis=1, inplace=True)

#Resetando index
df_final.set_index('Sourceid', inplace=True)
df_final.reset_index(inplace=True)

## Aplicação do filtro para export

In [186]:
print(config["filter_title"])
print(config["filter_year"])

data
2020


In [187]:
export = (config["type_save"]).upper()
filter_title = (config["filter_title"])
filter_year = (config["filter_year"])

if filter_year is not None and filter_title is not None:
    filter_title = filter_title.upper()
    df_filter = df_final[(df_final['year'] == str(filter_year)) & df_final['title'].str.upper().str.contains(filter_title)]

elif filter_year is not None:
    df_filter = df_final[df_final['year'] == str(filter_year)]
            
elif filter_title is not None:
    filter_title = filter_title.upper()
    df_filter = df_final[df_final['title'].str.upper().str.contains(filter_title)]
        
else:
    df_filter = df_final

## Export dos arquivos

In [188]:
if "CSV" in export or "JSON" in export or "YAML" or "PARQUET" in export:    
    ##CSV
    if "CSV" in export:
        df_filter.to_csv(config["csv_dest"], sep='|', encoding='utf-8', header='true')
        print("Export para csv realizado com sucesso!")
    
    ##JSON
    if "JSON" in export: 
        df_filter.to_json(config["json_dest"], orient='records')
        print("Export para json realizado com sucesso!")
     
    ##YAML        
    if "YAML" in export: 
        export_yaml = yaml.dump(
            df_filter.to_dict(orient='records'),
                    sort_keys=False, default_flow_style=None)
            
        with open(config["yaml_dest"], 'w') as f:
            f.write(export_yaml)
        print("Export para yaml realizado com sucesso!")
    
    ##PARQUET        
    if "PARQUET" in export:
        df_parquet = df_filter.astype(str)
        df_parquet.to_parquet(config["parquet_gzip_dest"],compression='gzip')
        print("Export para parquet realizado com sucesso!")
        
##ERRO - formato de export invalido        
else:
    print(f'O formato \'{export}\' solicitado não existe!\nOs formatos disponíveis são: csv, json, yaml e parquet')

Export para csv realizado com sucesso!
Export para json realizado com sucesso!
Export para yaml realizado com sucesso!
Export para parquet realizado com sucesso!


## Export para SQL Server

In [189]:
#Lê as configurações de conexão
server   = config["server"]
database = config["database"]
username = config["username"]
password = str(config["password"])

params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                     "SERVER="+server+";"
                                     "DATABASE="+database+";"
                                     "UID="+username+";"
                                     "PWD="+password+";")

#Cria a conexão com o banco de dados
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

#Especifica as colunas que serão utilizadas
data = pd.DataFrame(df_filter, columns = ['Sourceid', 'title', 'type_publication', 
            'issn', 'SJR', 'SJR_Best_Quartile', 'H_index', 'Total_Docs_2020', 
            'Total_Docs_3years', 'Total_Refs', 'Total_Cites_3years', 
            'Citable_Docs_3years', 'Cites_Doc_2years', 'Ref_Doc', 'Country', 
            'Region', 'publisher1', 'Coverage', 'Categories', 'Total_Cites', 
            'Journal_Impact_Factor', 'Eigenfactor_Score', 'series', 'location', 
            'keywords', 'numpages', 'pages', 'booktitle', 'abstract', 'doi', 
            'url', 'address', 'isbn', 'year', 'author', 'ID', 'articleno', 
            'month', 'journal', 'number', 'volume', 'issue_date', 'note', 
            'edition', 'editor', 'rank', 'content_type', 'article_number',
            'pdf_url', 'html_url', 'abstract_url', 'publication_title', 
            'conference_location', 'conference_dates', 'publication_number',
            'is_number', 'publication_date', 'start_page', 'end_page', 
            'citing_paper_count', 'citing_patent_count', 'issue', 'affiliation', 
            'authorUrl', 'id_author', 'author_order', 'load_date', 'creator'
            'publication_name', 'cover_date', 'starting_page', 'pii', 'ending_page'])            

#Exporta para a tabela no SQL
data.to_sql("tb_exportdb", engine, schema="dbo", if_exists="append", index=False)

#Fecha a conexão
engine.dispose()